In [4]:
import pyrosetta; pyrosetta.init()
from pyrosetta import *
init()
from pyrosetta.toolbox import pose_from_rcsb
from pyrosetta.toolbox import cleanATOM
from pyrosetta.teaching import *
from pyrosetta import PyMOLMover
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from split_energy import energy_intervals
from split_energy import split_energy
from split_energy import plot
import IPython
from stride import stride
from stride import no_secondary

┌──────────────────────────────────────────────────────────────────────────────┐
│                                 PyRosetta-4                                  │
│              Created in JHU by Sergey Lyskov and PyRosetta Team              │
│              (C) Copyright Rosetta Commons Member Institutions               │
│                                                                              │
│ NOTE: USE OF PyRosetta FOR COMMERCIAL PURPOSES REQUIRE PURCHASE OF A LICENSE │
│         See LICENSE.PyRosetta.md or email license@uw.edu for details         │
└──────────────────────────────────────────────────────────────────────────────┘
PyRosetta-4 2024 [Rosetta PyRosetta4.Release.python312.ubuntu 2024.42+release.3366cf78a3df04339d1982e94531b77b098ddb99 2024-10-11T08:24:04] retrieved from: http://www.pyrosetta.org
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.Release.python312.ubuntu r388 2024.42+release.3366cf78a3 3366cf78a3

In [5]:
ifp, se_ifp = split_energy("1XO0")
plot(se_ifp)
merged_ifp, filtered_ifp = energy_intervals(se_ifp)
plot(se_ifp, merged_intervals=merged_ifp, filtered_intervals=filtered_ifp, type ='intervals')

core.import_pose.import_pose: [ ERROR ] File: 1XO0 not found!



ERROR: Cannot open file "1XO0"
ERROR:: Exit from: /home/benchmark/rosetta/source/src/core/import_pose/import_pose.cc line: 361


RuntimeError: 

File: /home/benchmark/rosetta/source/src/core/import_pose/import_pose.cc:361
[ ERROR ] UtilityExitException
ERROR: Cannot open file "1XO0"



In [ ]:
ifp.total_residue()

In [ ]:
gfp, se_gfp = split_energy("1GFL")
plot(se_gfp)
merged_gfp, filtered_gfp = energy_intervals(se_gfp)
plot(se_gfp, merged_intervals=merged_gfp, filtered_intervals=filtered_gfp, type ='intervals')

In [ ]:
ubq, se_ubq = split_energy("1UBQ")
plot(se_ubq)
merged_ubq, filtered_ubq = energy_intervals(se_ubq)
plot(se_ubq, merged_intervals=merged_ubq, filtered_intervals=filtered_ubq, type ='intervals')

In [ ]:
lac, se_lac = split_energy("1ZG4")
plot(se_lac)
merged_lac, filtered_lac = energy_intervals(se_lac)
plot(se_lac, merged_intervals=merged_lac, filtered_intervals=filtered_lac, type ='intervals')

In [ ]:
pdb = '/home/hd/hd_hd/hd_ot299/switchable-split-proteins/data/1zg4.pdb'
temp = stride(pdb, system='mac', stride_dir='/home/hd/hd_hd/hd_ot299/.conda/envs/main/bin/stride', output_dir='/data/strideout')
print(temp)
temp[['resnum', 'saa']] = temp[['resnum', 'saa']].astype('float64')


In [ ]:
split_sites = []
for i in range(1,lac.total_residue()):
    sites = (i, i+1)
    if (temp.loc[temp['resnum']==sites[0], 'saa'].values[0] > 30) and (temp.loc[temp['resnum']==sites[1], 'saa'].values[0] > 30):
        ### If-condition for conservation score: < 2
        if (temp.loc[temp['resnum']==sites[0], 'code'].isin(['C','T']).iloc[0]) | (temp.loc[temp['resnum']==sites[1], 'code'].isin(['C','T']).iloc[0]):
            split_sites.append(sites)

In [ ]:
loops = []
start = None
for i in range(1,lac.total_residue()+1):
    if (temp.loc[temp['resnum']==i, 'code'].isin(['C','T']).iloc[0]):
        if start is None:
            start = i
    else:
        if start is not None:
            loops.append((start, i-1))
            start = None
if start is not None:
    loops.append((start, i))

In [ ]:
dump_list = []
sites_loop = []
for start, end in loops:
    for i in split_sites:
        if start <= i[0]+1 <= end+1:
            dump_list.append(i)
        else:
            sites_loop.append(dump_list)
            dump_list = []
sites_loop.append(dump_list)
sites_loop = [x for x in sites_loop if x != []]

In [ ]:
sites_loop_filtered = []
for i in sites_loop:
    list_sum = []
    if len(i) > 1:
        for site in i:
            sum = temp.loc[temp['resnum']==site[0], 'saa'].values[0] + temp.loc[temp['resnum']==site[1], 'saa'].values[0]
            list_sum.append(sum)
        sorted = list(np.argsort(list_sum)[::-1][:2])
        if np.abs(i[sorted[0]][0] - i[sorted[1]][0]) > 5:
            sites_loop_filtered.append([i[j] for j in sorted])
        else:
            sites_loop_filtered.append([i[sorted[0]]])
    else:
        sites_loop_filtered.append(i)

In [ ]:
for i in sites_loop_filtered[:]:
    bool = []
    if len(i) < 2:
        for start, end in filtered_lac:
            if not (start <= i[0][0] + 1 <= end + 1):
                bool.append(False)
            else:
                bool.append(True)
        if not any(bool):
            sites_loop_filtered.remove(i)

In [6]:
from split_sites import split_sites_func

split_sites_func('hd_la299', '1zg4')



┌──────────────────────────────────────────────────────────────────────────────┐
│                                 PyRosetta-4                                  │
│              Created in JHU by Sergey Lyskov and PyRosetta Team              │
│              (C) Copyright Rosetta Commons Member Institutions               │
│                                                                              │
│ NOTE: USE OF PyRosetta FOR COMMERCIAL PURPOSES REQUIRE PURCHASE OF A LICENSE │
│         See LICENSE.PyRosetta.md or email license@uw.edu for details         │
└──────────────────────────────────────────────────────────────────────────────┘
PyRosetta-4 2024 [Rosetta PyRosetta4.Release.python312.ubuntu 2024.42+release.3366cf78a3df04339d1982e94531b77b098ddb99 2024-10-11T08:24:04] retrieved from: http://www.pyrosetta.org
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.Release.python312.ubuntu r388 2024.42+release.3366cf78a3 3366cf78a3

TypeError: split_sites_func() takes 1 positional argument but 2 were given

In [ ]:
def split_sites_func(user,pdb):
    pdb = f'./data/{pdb}.pdb'
    temp = stride(pdb, system='mac', stride_dir='./stride', output_dir='/data/strideout')
    temp[['resnum', 'saa']] = temp[['resnum', 'saa']].astype('float64')
    pose, se_pose = split_energy(pdb)
    merged_pose, filtered_pose = energy_intervals(se_pose)
    split_sites = []
    for i in range(1,pose.total_residue()):
        sites = (i, i+1)
        if (temp.loc[temp['resnum']==sites[0], 'saa'].values[0] > 30) and (temp.loc[temp['resnum']==sites[1], 'saa'].values[0] > 30):
        ### If-condition for conservation score: < 2
            if (temp.loc[temp['resnum']==sites[0], 'code'].isin(['C','T']).iloc[0]) | (temp.loc[temp['resnum']==sites[1], 'code'].isin(['C','T']).iloc[0]):
                split_sites.append(sites)
    loops = []
    start = None
    for i in range(1,pose.total_residue()+1):
        if (temp.loc[temp['resnum']==i, 'code'].isin(['C','T']).iloc[0]):
            if start is None:
                start = i
        else:
            if start is not None:
                loops.append((start, i-1))
                start = None
    if start is not None:
        loops.append((start, i))
    loops = []
    start = None
    for i in range(1,pose.total_residue()+1):
        if (temp.loc[temp['resnum']==i, 'code'].isin(['C','T']).iloc[0]):
            if start is None:
                start = i
        else:
            if start is not None:
                loops.append((start, i-1))
                start = None
    if start is not None:
        loops.append((start, i))
    dump_list = []
    sites_loop = []
    for start, end in loops:
        for i in split_sites:
            if start <= i[0]+1 <= end+1:
                dump_list.append(i)
            else:
                sites_loop.append(dump_list)
                dump_list = []
    sites_loop.append(dump_list)
    sites_loop = [x for x in sites_loop if x != []]
    sites_loop_filtered = []
    for i in sites_loop:
        list_sum = []
        if len(i) > 1:
            for site in i:
                sum = temp.loc[temp['resnum']==site[0], 'saa'].values[0] + temp.loc[temp['resnum']==site[1], 'saa'].values[0]
                list_sum.append(sum)
            sorted = list(np.argsort(list_sum)[::-1][:2])
            if np.abs(i[sorted[0]][0] - i[sorted[1]][0]) > 5:
                sites_loop_filtered.append([i[j] for j in sorted])
            else:
                sites_loop_filtered.append([i[sorted[0]]])
        else:
            sites_loop_filtered.append(i)
    for i in sites_loop_filtered[:]:
        bool = []
        if len(i) < 2:
            for start, end in filtered_pose:
                if not (start <= i[0][0] + 1 <= end + 1):
                    bool.append(False)
                else:
                    bool.append(True)
            if not any(bool):
                sites_loop_filtered.remove(i)
    return sites_loop_filtered